In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

Dans le but de manipuler et analyser efficacement nos données, nous avons importer la bibliotheque **pandas** de python qui s'appuie sur la structure des données.

---



Le module **numpy** va nous permettre de conserver nos données sous forme matricielle afin de faciliter les calculs vectoriels.

---



---



Nous utilisons le module matplotlib.pyplot pour visualiser nos données sous différent aspects avec des graphes

---



---



Le module scikit learn (sklearn) est le module le plus populaire utiliser en machine learning. Dans notre cadre, nous faisons appelle à **sklearn.metrics** qui permet de calculer la performance de nos modèles.

---



---



In [2]:
pd.options.mode.chained_assignment = None

"""
la methode options.mode.chained_assignment de pandas nous permet d'arrêter les avertissement relatif à la copie de données répétée dans
notre dataframe.

"""

"\nla methode options.mode.chained_assignment de pandas nous permet d'arrêter les avertissement relatif à la copie de données répétée dans\nnotre dataframe.\n\n"

**MODELE 1 : SIMPLE OVERALL AVARAGE MODEL**

In [3]:
#Importation de nos dataset d'entrainement et de test

df_user_data = pd.read_excel("C:/Users/asus/Desktop/Datasets Cooding-Week/Livrable_code_g_colab/datasets_excel/user1_base.xlsx")
df_user_data_test = pd.read_excel("C:/Users/asus/Desktop/Datasets Cooding-Week/Livrable_code_g_colab/datasets_excel/user1_test.xlsx") 

In [4]:
# Ettiqueter nos colonnes

headers = ["userid", "itemid", "rating", "timestamp"]
df_user_data.columns = headers
df_user_data_test.columns = headers

In [5]:
# on va creer une matrice R de taille 943x1682 pour enregistrer
# les notes des films notés par les utilisateurs

R = np.zeros((943, 1682))
zeros =np.zeros((943, 1))

for i in range(0, 80000):
    n = df_user_data["userid"][i]
    m = df_user_data["itemid"][i]
    c = df_user_data["rating"][i]
    R[n-1][m-1]= c

In [21]:
# le code suivant va nous permettre de calculer la moyenne globale de tous les films 
# notés par les utilisateurs

x_counts = 0
total = 0
for j in range(0, 1682):
    for i in range(0, 943):
        total += R[i][j]
        if R[i][j] ==0 :
            x_counts+=1

moyenne_mu = total / (943*1682 - x_counts)
moyenne_mu

3.52835

In [22]:
# On enregistre la moyenne globale de tous les films dans une nouvelle colonne 
# du dataset 

df_user_data_test['mean_rating'] = moyenne_mu
df_user_data_test.head()

,userid,itemid,rating,timestamp,mean_rating
0,1,6,5,887431973,3.52835
1,1,10,3,875693118,3.52835
2,1,12,5,878542960,3.52835
3,1,14,5,874965706,3.52835
4,1,17,3,875073198,3.52835




---




In [23]:
# Calcul du RMSE pour le modèle 1

rmse_1 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating'])
rmse_1

1.3309681924999999

**INTERPRETATION :** La valeur du ***rmse_1*** est bien trop grande. Cela nous indique que notre modèle commet beaucoup trop d'erreur. Il faut donc trouver un modèle plus performent qui reduit les erreurs.

---



---



---



**MODELE 2 : MOVIE EFFECT MODEL**

---



---



In [24]:
# CREATION D'UNE LISTE CONTENANT LES BIAIS DE CHAQUE FILMS EN PRNANT EN COMPTE LES FILMS N'AYANT PAS ÉTÉ NOTÉS

bias = []

for i in range(0, 1682):
    
    somme = 0
    count = 0
    for j in range(0, 943):
        if R[j][i] != 0:
            count += 1
            somme  += R[j][i]
    if count != 0:
        b = (somme / count) - moyenne_mu
        bias.append(b)

    else:
        b = 0
        bias.append(b)

In [25]:
# Creation de la colonne mean_rating_2 dans le dataset remplie de 0  

predicted_rating_2 = list(bias + moyenne_mu )

df_user_data_test['mean_rating_2'] = 0

In [26]:
# Completion de mean_rating_2 dans le dataset
for i in range(len(df_user_data_test["itemid"])):
    df_user_data_test["mean_rating_2"][i]= predicted_rating_2[df_user_data_test["itemid"][i]-1]

In [27]:
# CALCUL DU ROOT MEAN SQUARE ERROR 
rmse_2 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating_2'])
rmse_2

1.0679390625704295

**INTERPRETATIONS :** La prise en compte du biais de chaque film rend notre modèle de plus en plus précis. Bien que le RMSE ait diminué, il peut encore être reduit en prenant aussi en compte le biais de chaque utilisateur.

---



---



 **MODELE 3 : Movie and user effect multi-variate model**

In [28]:
# CREATION D'UNE LISTE CONTENANT LES BIAIS DE CHAQUE UTILISATEUR
bias_u = []

for j in range(0, 943):
    
    somme = 0
    count = 0
    for i in range(0, 1642):
        if R[j][i] != 0:
            count += 1
            somme  += R[j][i] - moyenne_mu - bias[i]
    if count != 0:
        b_u = (somme / count)
        bias_u.append(b_u)

In [29]:
R_predicted_rating_3 = np.zeros((943, 1682)) # ICI, nous avons maintenant deux facteurs a prendre en compte (le biais relatif a chacun des films et celui de chaque utilisateurs), ce 
                                              # qui justifie la creation d'un tableau a deux dimensions

In [30]:
# On enregistre les valeurs des biais de chaque utilisateur par rapport à chaque film dans la matrice R

for i in range(0, 943):
    for j in range(0, 1482):
        R_predicted_rating_3[i][j]= moyenne_mu + bias[j] + bias_u[i]

In [33]:
# Création et Completion de mean_rating_3 que dans le dataset

df_user_data_test['mean_rating_3'] = 0

for i in range(len(df_user_data_test["itemid"])):
    df_user_data_test["mean_rating_3"][i]= R_predicted_rating_3[df_user_data_test["userid"][i]-1][df_user_data_test["itemid"][i]-1]

In [34]:
rmse_3 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating_3'])
rmse_3

0.9413617020803795

**INTERPRETATION :** Comme on peut le constater, les erreurs de notre modèle ont encore baissées. Ce qui rend notre modèle 3 plus performant que les modèles précédents.  

---



Une autre méthode pour reduire améliorer la performance de notre modèle est de réduire les biais utilisateurs et ceux des films qui sont élevés afin de garantir une certaine harmonisation dans les notes des films (Modèle 4)

---



---




**MODELE 4 : Regularized movie and user effect model**

In [37]:
# liste des rmse en fonction d#un facteur de reduction de biais que nous choisissons de manière aléatoire 
# à travers lambdas

rmses = []

lambdas = np.arange(1.0, 1.50, 0.10)

R_predicted_rating_4 = np.zeros((943, 1682))

In [38]:
# Que represente l
for l in lambdas:
    mu = np.mean(df_user_data['rating'])

    #b_i = df_user_data.groupby('itemid').agg(b_i=('rating', lambda x: np.sum(x - mu) / (len(x) + l)))

    b_i = []
    # Definition de b_i
    for i in range(0, 1682):
        
        somme = 0
        count = 0
        for j in range(0, 943):
            if R[j][i] != 0:
                count += 1
                somme  += R[j][i]
        if count != 0:
            b = (somme - moyenne_mu*count )/ (count + l)
            b_i.append(b)

        else:
            b = 0
            b_i.append(b)

    b_u2 = []

    for j in range(0, 943):
        
        somme = 0
        count = 0
        for i in range(0, 1642):
            if R[j][i] != 0:
                count += 1
                somme  += (R[j][i] - moyenne_mu - bias[i])
        if count != 0:
            b = (somme / count)
            b_u2.append(b)


    for i in range(0, 943):
        for j in range(0, 1482):
            R_predicted_rating_4[i][j]= moyenne_mu + b_i[j] + b_u2[i]
    #print(R_predicted_rating_3)

    df_user_data_test['mean_rating_4'] = 0

    for i in range(len(df_user_data_test["itemid"])):
        df_user_data_test["mean_rating_4"][i]= R_predicted_rating_4[df_user_data_test["userid"][i]-1][df_user_data_test["itemid"][i]-1]

    

    rmses.append(sklearn.metrics.mean_squared_error(df_user_data_test["mean_rating_4"], df_user_data_test['rating']))
    df_user_data_test['b_i'] = 0 
    df_user_data_test['b_u'] = 0 


    #Definition et completion de mean_rating_2 dans le dataset
    for i in range(len(df_user_data_test["itemid"])):
        df_user_data_test["b_i"][i]= b_i[df_user_data_test["itemid"][i]-1]
        df_user_data_test["b_u"][i]= b_u2[df_user_data_test["userid"][i]-1]

In [19]:
df_user_data_test.head()

,userid,itemid,rating,timestamp,mean_rating,mean_rating_2,mean_rating_3,mean_rating_4,b_i,b_u
0,1,6,5,887431973,3.52835,0,3.508952,3.517349,-0.119953,0.108952
1,1,10,3,875693118,3.52835,0,3.985664,3.979109,0.341807,0.108952
2,1,12,5,878542960,3.52835,0,4.507056,4.501324,0.864021,0.108952
3,1,14,5,874965706,3.52835,0,4.008952,4.005272,0.367970,0.108952
4,1,17,3,875073198,3.52835,0,3.289508,3.296141,-0.341161,0.108952


In [20]:
rmses

[0.9373868562951772,
 0.9373258715698437,
 0.9372917484245303,
 0.9372809497892511,
 0.9372905337620469]

In [39]:
rmses.sort() # on trie les valeurs des rmse par ordre croissant 


In [41]:
rmse_4 = rmses[0] # on récupère le rmse le plus faible pour le modèle 4
rmse_4

0.9372809497892511